In [83]:
pip install -r requirements.txt

     ------------------------------------ 139.8/139.8 kB 689.2 kB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 214.0/214.0 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Config

In [4]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7,
)

def ask(prompt: str):
    return llm.invoke([HumanMessage(content=prompt)]).content




## Zero-Shot

In [3]:
# Langkah 1: Buat prompt Zero-Shot
prompt_1 = "Apa itu Prompt Engineering dalam konteks AI?"

# Langkah 2: Kirim prompt ke model
response_1 = ask(prompt_1)

# Langkah 4: Tampilkan hasil
print("Jawaban 1:\n", response_1)

Jawaban 1:
 Prompt engineering, dalam konteks AI, adalah proses **merancang dan mengoptimalkan teks masukan (prompt) yang diberikan kepada model AI (terutama Large Language Models atau LLMs) untuk menghasilkan keluaran yang diinginkan.**  Sederhananya, ini adalah seni dan ilmu untuk "berbicara" dengan AI agar mendapatkan jawaban atau hasil yang paling akurat, relevan, dan bermanfaat.

**Mengapa Prompt Engineering Penting?**

* **Mengontrol Keluaran:** Prompt yang dirancang dengan baik dapat membimbing model AI untuk menghasilkan jawaban yang lebih spesifik, akurat, dan sesuai dengan kebutuhan pengguna.
* **Meningkatkan Kualitas:** Prompt yang buruk dapat menghasilkan jawaban yang tidak relevan, tidak akurat, atau bahkan berbahaya. Prompt engineering membantu meningkatkan kualitas keluaran secara signifikan.
* **Membuka Potensi Model:** Dengan prompt yang tepat, kita dapat membuka potensi tersembunyi dari model AI dan memanfaatkannya untuk berbagai tugas yang kompleks, seperti menulis k

## Few-Shot  *perbaiki

In [4]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [ ]:
dataset = [
    {"input": "Meeting agenda for Monday's project sync", "label": "Work"},
    {"input": "Get 3 pizzas for the price of 1 – today only!", "label": "Promotion"},
    {"input": "Mom sent you a recipe – check it out!", "label": "Personal"},
    {"input": "You've been selected for a $1000 gift card. Click here!", "label": "Spam"},
]

In [6]:
example_prompt = PromptTemplate(
    input_variables=["input", "label"],
    template="Email: {input}\nCategory: {label}\n"
)

In [9]:
few_shot_prompt = FewShotPromptTemplate(
    examples=dataset,
    example_prompt=example_prompt,
    suffix="Email: {input}\nCategory:",
    input_variables=["input"]
)

In [11]:
test_input = "Free vacation to the Bahamas! Just confirm your email now."

formatted_prompt = few_shot_prompt.format(input=test_input)
print(ask(formatted_prompt))

Spam


In [12]:
test_input = "Pizza hut discount 50% for easter only in Jogja."

formatted_prompt = few_shot_prompt.format(input=test_input)
print(ask(formatted_prompt))

Promotion


In [39]:
email = """Dear All,

We are excited to welcome you to Siloam AI team! Please meet the team:

Date: Monday, April 21st 2025
Time: 1:30 PM - end
Location: online via Gmeet

Looking forward to your participation and collaboration. 

Best regards, 
Dira
Recruitment Team | PT. BitHealth Teknologi Pintar
When
Monday Apr 21, 2025 ⋅ 1:30pm – 3pm (Western Indonesia Time - Jakarta)
Organizer
Nurul Undira"""

formatted_prompt = few_shot_prompt.format(input=email)
print(ask(formatted_prompt))

Work


## Self-Consistency  *perbaiki


In [18]:
from collections import Counter

prompt_template = """
You are a helpful assistant that classifies emails. Your task is to read the email and decide if it's:
- Personal
- Work
- Promotion
- Spam

Email: "{email}"

Let's think step-by-step:
"""

In [23]:

email_input = "Get 3 pizzas for the price of 1 – today only!"

prompt = prompt_template.format(email=email_input)

In [ ]:
# Lakukan sampling beberapa kali
results = [ask(prompt) for _ in range(5)]
# Tampilkan hasilnya
print("\n--- All Answers ---\n")
for i, r in enumerate(results, 1):
    print(f"[{i}] {r.strip()}\n")


--- All Answers ---

[1] The email is advertising a special offer on pizzas. This is a marketing tactic used to attract customers.

Therefore, the email is a **Promotion**.

[2] The email is advertising a special offer on pizzas. This is a marketing tactic used to attract customers.

Therefore, the email is a promotion.

Final Answer: Promotion

[3] The email is advertising a special offer on pizzas. This falls under marketing and promotional content.

Therefore, the classification is:
**Promotion**

[4] The email advertises a special offer on pizzas for a limited time. This is a marketing tactic used to attract customers.

Therefore, the email is a **Promotion**.

[5] The email is advertising a deal on pizzas. This is a marketing tactic used to attract customers.

Therefore, the email is a **Promotion**.



In [26]:

labels = [r.strip().split()[-1].replace(".", "") for r in results]
most_common = Counter(labels).most_common(1)[0][0]

print(" final answer:", most_common)


 final answer: **Promotion**


In [36]:
email = """Dear All,

We are excited to welcome you to Siloam AI team! Please meet the team:

Date: Monday, April 21st 2025
Time: 1:30 PM - end
Location: online via Gmeet

Looking forward to your participation and collaboration. 

Best regards, 
Dira
Recruitment Team | PT. BitHealth Teknologi Pintar
When
Monday Apr 21, 2025 ⋅ 1:30pm – 3pm (Western Indonesia Time - Jakarta)
Organizer
Nurul Undira"""

prompt = prompt_template.format(email=email)

In [37]:
# Lakukan sampling beberapa kali
results = [ask(prompt) for _ in range(5)]
# Tampilkan hasilnya
print("\n--- All Answers ---\n")
for i, r in enumerate(results, 1):
    print(f"[{i}] {r.strip()}\n")


--- All Answers ---

[1] The email is sent to multiple recipients ("Dear All"). It's from a recruitment team at a company. It welcomes new members to a team and provides details about a meeting. This clearly relates to work.

Therefore, the classification is:
**Work**

[2] The email is announcing a meeting to welcome someone to a team. It's coming from a recruitment team of a company. Therefore, it's a **Work** email.

[3] The email is announcing a welcoming event for new members of a team. It includes details about the date, time, and location of the event. The sender is from the recruitment team of a company.

Based on the above analysis, the email is most likely related to **Work**.

[4] The email is from a recruitment team welcoming someone to a team. This is related to work.

Therefore, the classification is:
Work

[5] The email is announcing a welcoming event for new members joining the Siloam AI team. It provides details about the date, time, and location of the event. The send

In [38]:

labels = [r.strip().split()[-1].replace(".", "") for r in results]
most_common = Counter(labels).most_common(1)[0][0]

print(" final answer:", most_common)


 final answer: **Work**


## Chain Of Thought

In [29]:
def classify_email_with_cot(email_text: str):
    cot_prompt = f"""
    You are an email classifier. Think through each step before deciding on the category.

    Classify the following email into one of the categories:
    - Personal
    - Work
    - Promotion
    - Spam

    Email: "{email_text}"

    Step-by-step reasoning:
    1. What is the email about?
    2. Who might be sending this kind of email?
    3. What is the likely purpose of the email?
    4. Based on this reasoning, which category fits best?

    Answer:
    """
    return ask(cot_prompt)


In [30]:
print(classify_email_with_cot("Don't miss our Black Friday deals – up to 80% off!"))


1.  The email is about Black Friday deals and discounts.
2.  Retail companies or businesses selling products/services are likely to send this type of email.
3.  The purpose is to promote sales and attract customers to make purchases.
4.  Therefore, the most appropriate category is **Promotion**.


In [35]:
email = """Dear All,

We are excited to welcome you to Siloam AI team! Please meet the team:

Date: Monday, April 21st 2025
Time: 1:30 PM - end
Location: online via Gmeet

Looking forward to your participation and collaboration. 

Best regards, 
Dira
Recruitment Team | PT. BitHealth Teknologi Pintar
When
Monday Apr 21, 2025 ⋅ 1:30pm – 3pm (Western Indonesia Time - Jakarta)
Organizer
Nurul Undira"""

print(classify_email_with_cot(email))

1.  **What is the email about?** The email is a welcome message to new members of the Siloam AI team, providing details about an introductory meeting.

2.  **Who might be sending this kind of email?** The email is sent from the recruitment team of a company named PT. BitHealth Teknologi Pintar.

3.  **What is the likely purpose of the email?** The purpose is to inform new team members about their welcome meeting and encourage their participation.

4.  **Based on this reasoning, which category fits best?** The email is clearly related to onboarding new employees at a company. Therefore, it falls under the **Work** category.

Answer: Work


## Tree of Thought  *perbaiki

In [32]:
def classify_email_with_tot(email_text: str):
    prompt = f"""
You are an intelligent system trained to think through problems in multiple ways.

You will explore multiple possible classifications of the email below.
Then, you will decide which classification is best.

Email: "{email_text}"

Think of 3 different ways to classify this email:
Thought 1:
- What is the email about?
- Who sent it?
- Why was it sent?
- What category might fit?

Thought 2:
- Try an alternate perspective or interpretation.

Thought 3:
- Consider another possible context or sender.

Now compare the three and decide which one makes the most sense.

Final decision (only one): Personal, Work, Promotion, or Spam

Answer:
"""
    return ask(prompt)


In [33]:
email = "Your package has shipped! Track your order here."
print(classify_email_with_tot(email))


Okay, let's analyze this email and classify it.

**Thought 1: The Obvious Interpretation**

*   **What is the email about?** A package shipment notification.
*   **Who sent it?** Likely a retailer or shipping company.
*   **Why was it sent?** To inform the recipient that their order has been shipped and provide tracking information.
*   **What category might fit?** This leans heavily towards a *Promotion* classification, as it's directly related to a purchase and encourages further engagement (tracking the package, potentially leading to future purchases). It could also be argued as *Personal* because it relates to something that was bought, but the actual content is more transactional.

**Thought 2: Alternate Perspective - Security Risk**

*   This email could be a phishing attempt. The link might lead to a fake tracking website designed to steal login credentials or install malware. The sender could be a malicious actor impersonating a legitimate company. In this case, the email woul

In [34]:
email = """Dear All,

We are excited to welcome you to Siloam AI team! Please meet the team:

Date: Monday, April 21st 2025
Time: 1:30 PM - end
Location: online via Gmeet

Looking forward to your participation and collaboration. 

Best regards, 
Dira
Recruitment Team | PT. BitHealth Teknologi Pintar
When
Monday Apr 21, 2025 ⋅ 1:30pm – 3pm (Western Indonesia Time - Jakarta)
Organizer
Nurul Undira"""

print(classify_email_with_tot(email))


Okay, let's analyze this email and classify it.

**Thought 1: Direct Interpretation**

*   **What is the email about?** It's an invitation to meet the Siloam AI team.
*   **Who sent it?** Dira from the Recruitment Team at PT. BitHealth Teknologi Pintar.
*   **Why was it sent?** To welcome new members to the team and introduce them.
*   **What category might fit?** This seems like a **Work** related email. It's internal communication within a company to onboard new employees.

**Thought 2: Alternate Perspective**

*   Perhaps it is a recruitment email. While it does not explicitly offer a job, it is welcoming someone to the team. This could be considered a **Promotion** of the company and its AI team, to encourage the recipient to accept a position or engage further.

**Thought 3: Considering Another Context**

*   What if the recipient didn't apply for a job at Siloam AI? If the recipient did not apply, the email could be unsolicited. In this case, it could be considered **Spam**. Howe

## Automatic Prompt Engineer

In [63]:
email = """Dear All,

We are excited to welcome you to Siloam AI team! Please meet the team:

Date: Monday, April 21st 2025
Time: 1:30 PM - end
Location: online via Gmeet

Looking forward to your participation and collaboration. 

Best regards, 
Dira
Recruitment Team | PT. BitHealth Teknologi Pintar
When
Monday Apr 21, 2025 ⋅ 1:30pm – 3pm (Western Indonesia Time - Jakarta)
Organizer
Nurul Undira"""

In [ ]:
def generate_prompt_candidates(n=3):
    prompt = f"""
You are an expert prompt engineer.

Task: Create {n} different prompts to classify an email into one of the following categories:
- Personal
- Work
- Promotion
- Spam

Each prompt should guide the model to think before answering, include instructions if needed, and allow classification of complex real-world emails.

Use "{{email}}" as placeholder for the input email.

Format:
PROMPT 1: ...
PROMPT 2: ...
PROMPT 3: ...
"""
    return ask(prompt)



In [67]:
# 3. Ekstrak prompt individual dari teks yang dihasilkan
def extract_prompts(candidates_text, n=3):
    prompt_list = []
    for i in range(1, n + 1):
        start = f"PROMPT {i}:"
        end = f"PROMPT {i+1}:" if i < n else None
        section = candidates_text.split(start)[1]
        prompt = section.split(end)[0].strip() if end else section.strip()
        prompt_list.append(prompt)
    return prompt_list

In [68]:
# 4. Evaluasi prompt terhadap email
def evaluate_prompts(prompt_list, test_email):
    results = []
    for i, prompt_template in enumerate(prompt_list, 1):
        filled_prompt = prompt_template.replace("{email}", test_email)
        print(f"\n================= PROMPT {i} =================\n{prompt_template}")
        print(f"\n➡️ Filled Prompt:\n{filled_prompt}")

        output = ask(filled_prompt)
        print(f"\n🧠 Model Output:\n{output.strip()}")
        results.append((f"PROMPT {i}", output.strip()))
    return results

In [69]:
# 5. Analisis hasil dengan voting label terbanyak
def analyze_results(results):
    labels = [r[1].split()[-1].replace(".", "") for r in results]
    most_common = Counter(labels).most_common(1)[0]
    return most_common, labels

In [70]:
# Step A: Generate prompt candidates
candidate_text = generate_prompt_candidates(n=3)

# Step B: Parse jadi list of prompt strings
prompt_list = extract_prompts(candidate_text)

# Step C: Uji semua prompt ke email Siloam
results = evaluate_prompts(prompt_list, email)

# Step D: Ambil label terbanyak
most_common_label, all_labels = analyze_results(results)

print(f"\n✅ Most consistent label: {most_common_label[0]} ({most_common_label[1]} votes)")



================= PROMPT 1 =================
**

First, analyze the email provided below and identify key indicators such as sender, recipient, subject line, keywords, and the overall tone. Consider the context: Is this email from a known contact? Does it relate to your professional responsibilities? Does it advertise a product or service? Does it contain suspicious links or requests? Classify the email into one of the following categories: Personal, Work, Promotion, or Spam. Justify your classification with specific reasons based on your analysis.

Email: {email}

Classification (choose one): Personal, Work, Promotion, Spam
Justification:
**

➡️ Filled Prompt:
**

First, analyze the email provided below and identify key indicators such as sender, recipient, subject line, keywords, and the overall tone. Consider the context: Is this email from a known contact? Does it relate to your professional responsibilities? Does it advertise a product or service? Does it contain suspicious links

## Program-Aided Language Models

In [ ]:
prompt = """
You are a Python expert.

Write a Python function called `solve()` that calculates the kinetic energy
of an object with mass = 75 kg and velocity = 20 m/s.

Use the formula: Ek = 0.5 * mass * velocity^2

Just output the function code.
"""

In [78]:
generated_code = ask(prompt)

print("🧠 Generated Code:\n")
print(generated_code)



🧠 Generated Code:

```python
def solve():
    """Calculates the kinetic energy of an object.

    Returns:
        float: The kinetic energy of the object in Joules.
    """
    mass = 75  # kg
    velocity = 20  # m/s
    kinetic_energy = 0.5 * mass * (velocity ** 2)
    return kinetic_energy
```


In [79]:
def solve():
    """Calculates the kinetic energy of an object.

    Returns:
        float: The kinetic energy of the object in Joules.
    """
    mass = 75  # kg
    velocity = 20  # m/s
    kinetic_energy = 0.5 * mass * (velocity ** 2)
    return kinetic_energy

In [81]:
hasil = solve()
print(f"\n✅ Kinetic Energy: {hasil} Joules")


✅ Kinetic Energy: 15000.0 Joules


## Multimodal COT

In [2]:
import base64

# Path ke gambar lokal
image_path = r"C:\Users\Fadhil Wicaksono\Documents\Training\Prompting-Langchain\Sepatu.png"

# Encode gambar jadi base64
def encode_image(filepath):
    with open(filepath, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

image_data = encode_image(image_path)


In [7]:
def ask_multimodal(text_prompt: str, image_base64: str):
    messages = [
        HumanMessage(
            content=[
                {"type": "text", "text": text_prompt},
                {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_base64}"}},
            ]
        )
    ]
    return llm.invoke(messages).content


In [10]:
multimodal_prompt = (
    "Bagus engga ini buat naik gunung merbabu untuk trail run kurang lebih 3000 mdpl, dan jenis sepatu apa yang digambar hokanya ? "
)

result = ask_multimodal(multimodal_prompt, image_data)
print(result)

Mari kita bahas tentang sepatu Hoka untuk trail run di Merbabu (kurang lebih 3000 mdpl):

*   **Apakah Sepatu Hoka Bagus untuk Trail Run di Merbabu?**

    Secara umum, sepatu Hoka bisa menjadi pilihan yang baik untuk trail run, terutama karena bantalan (cushioning) yang tebal dan nyaman. Namun, apakah sepatu *ini* cocok untuk Merbabu, itu tergantung pada beberapa faktor:

    *   **Jenis Medan:** Merbabu memiliki medan yang bervariasi, mulai dari tanah, bebatuan, hingga akar pohon. Sepatu dengan *grip* (cengkraman) yang baik sangat penting. Perhatikan solnya. Apakah pola *lug* (tonjolan pada sol) cukup dalam dan agresif? Jika tidak, sepatu ini mungkin kurang ideal untuk medan yang licin atau curam.
    *   **Kondisi Fisik dan Pengalaman:** Jika Anda terbiasa dengan sepatu minimalis atau memiliki kaki yang kuat, Anda mungkin tidak memerlukan bantalan setebal Hoka. Namun, jika Anda mencari kenyamanan dan perlindungan ekstra, Hoka bisa menjadi pilihan yang baik.
    *   **Kondisi Cuaca:*